## Online News Popularity Prediction Using Deep Learning

#### Applied two deep learning models in my dataset, Single Layer Perceptron and Multi Layer Perceptron

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.decomposition import PCA as sklearnPCA
import warnings
warnings.filterwarnings("ignore")

In [5]:
import tensorflow as tf
from sklearn.neural_network import MLPClassifier

### Importing the dataset

In [7]:
data = pd.read_csv('OnlineNewsPopularity/OnlineNewsPopularity.csv')
data.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [8]:
def converter(x):
    if(x > 1400):
        return 1
    else:
        return 0

In [9]:
data['NumberOfShares'] = data[' shares'].apply(converter)

In [10]:
data.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,NumberOfShares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593,0
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711,0
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500,1
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200,0
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505,0


## Data Sampling

In [15]:
X = data.drop([' shares', 'NumberOfShares', 'url'], axis=1)
y = data['NumberOfShares']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

### Data Cleanup

In [16]:
data = data[data[' n_unique_tokens'] <= 1] 
data = data[data[' n_non_stop_words'] <= 1]
data = data[data[' n_non_stop_unique_tokens'] <= 1]

In [17]:
data = data[data[' rate_positive_words'] + data[' rate_negative_words'] != 0]

In [18]:
data = data[data[' average_token_length'] != 0]

In [19]:
data_week = data.iloc[:, 31:37]

In [20]:
data1 = data.drop(data.iloc[:, 31:37], axis=1)

### Resampling Data

In [21]:
X_new = data1.drop(['url', ' shares', 'NumberOfShares'], axis=1)

In [148]:
y_new = data1.iloc[:,-1:]

In [151]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.30)

In [152]:
X_new.head()

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
0,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,0.0,...,0.378636,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500
1,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,0.0,...,0.286915,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000
2,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,0.0,...,0.495833,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000
3,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,0.0,...,0.385965,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000
4,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,0.0,...,0.411127,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364


In [153]:
#clf = MLPClassifier(activation='logistic', alpha=0.5, batch_size=1000, hidden_layer_sizes=(30000,), learning_rate='adaptive'
#                   , max_iter = 1000)

### Normalize x_data values for better prediction

In [154]:
# Training Data

In [177]:
train_x = MinMaxScaler().fit_transform(X_train)
print("Training Data :", train_x.shape)

Training Data : (26920, 53)


In [117]:
# Testing Data

In [178]:
test_x = MinMaxScaler().fit_transform(X_test)
print("Testing Data :", test_x.shape)

Testing Data : (11538, 53)


In [179]:
train_y = y_train

In [180]:
test_y = y_test

### Make ANN-SLP Model

#### Make "Placeholder" for dinamic variable allocation

In [181]:
X = tf.placeholder(tf.float32, [None,53])
Y = tf.placeholder(tf.float32, [None, 1])

#### Make Weight, Bias value with randomly

In [182]:
# weight
W = tf.Variable(tf.random_normal([53,1], seed=0), name='weight')

In [183]:
# bias
b = tf.Variable(tf.random_normal([1], seed=0), name='bias')

### Make Output Results

In [184]:
logits = tf.matmul(X,W) + b

### Cross Entropy

In [185]:
hypothesis = tf.nn.sigmoid(logits)
cost_i = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,labels=Y)
cost = tf.reduce_mean(cost_i)

### Gradient Descent Optimizer

In [186]:
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

### Compare : original vs. prediction

In [187]:
prediction = tf.cast(hypothesis > 0.5, dtype=tf.float32)
correct_prediction = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))

### Activate Model

In [190]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(30001):
        sess.run(train, feed_dict={X: train_x, Y: train_y})
        if step % 1000 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={X: train_x, Y: train_y})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))
            
    train_acc = sess.run(accuracy, feed_dict={X: train_x, Y: train_y})
    test_acc,test_predict,test_correct = sess.run([accuracy,prediction,correct_prediction], feed_dict={X: test_x, Y: test_y})
    print("Model Prediction =", train_acc)
    print("Test Prediction =", test_acc)

Step:     0	Loss: 1.940	Acc: 50.78%
Step:  1000	Loss: 0.662	Acc: 61.62%
Step:  2000	Loss: 0.652	Acc: 62.67%
Step:  3000	Loss: 0.647	Acc: 63.05%
Step:  4000	Loss: 0.644	Acc: 63.24%
Step:  5000	Loss: 0.643	Acc: 63.50%
Step:  6000	Loss: 0.641	Acc: 63.64%
Step:  7000	Loss: 0.640	Acc: 63.75%
Step:  8000	Loss: 0.639	Acc: 63.90%
Step:  9000	Loss: 0.639	Acc: 63.94%
Step: 10000	Loss: 0.638	Acc: 64.00%
Step: 11000	Loss: 0.638	Acc: 64.05%
Step: 12000	Loss: 0.637	Acc: 64.08%
Step: 13000	Loss: 0.637	Acc: 64.12%
Step: 14000	Loss: 0.637	Acc: 64.17%
Step: 15000	Loss: 0.636	Acc: 64.18%
Step: 16000	Loss: 0.636	Acc: 64.21%
Step: 17000	Loss: 0.636	Acc: 64.23%
Step: 18000	Loss: 0.636	Acc: 64.24%
Step: 19000	Loss: 0.635	Acc: 64.26%
Step: 20000	Loss: 0.635	Acc: 64.31%
Step: 21000	Loss: 0.635	Acc: 64.31%
Step: 22000	Loss: 0.635	Acc: 64.31%
Step: 23000	Loss: 0.635	Acc: 64.33%
Step: 24000	Loss: 0.635	Acc: 64.34%
Step: 25000	Loss: 0.635	Acc: 64.31%
Step: 26000	Loss: 0.635	Acc: 64.34%
Step: 27000	Loss: 0.635	Acc:

### MLP Model Summary and Compare

In [208]:
def ann_mlp():
    print("===========Data Summary===========")
    print("Training Data :", train_x.shape)
    print("Testing Data :", test_x.shape)

    X = tf.placeholder(tf.float32, [None,53])
    Y = tf.placeholder(tf.float32, [None, 1])
    
    # input
    W1 = tf.Variable(tf.random_normal([53,106], seed=0), name='weight1')
    b1 = tf.Variable(tf.random_normal([106], seed=0), name='bias1')
    layer1 = tf.nn.sigmoid(tf.matmul(X,W1) + b1)
    
    # hidden1
    W2 = tf.Variable(tf.random_normal([106,106], seed=0), name='weight2')
    b2 = tf.Variable(tf.random_normal([106], seed=0), name='bias2')
    layer2 = tf.nn.sigmoid(tf.matmul(layer1,W2) + b2)
    
    # hidden2
    W3 = tf.Variable(tf.random_normal([106,159], seed=0), name='weight3')
    b3 = tf.Variable(tf.random_normal([159], seed=0), name='bias3')
    layer3 = tf.nn.sigmoid(tf.matmul(layer2,W3) + b3)
    
    # output
    W4 = tf.Variable(tf.random_normal([159,1], seed=0), name='weight4')
    b4 = tf.Variable(tf.random_normal([1], seed=0), name='bias4')
    logits = tf.matmul(layer3,W4) + b4
    hypothesis = tf.nn.sigmoid(logits)
    
    cost_i = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,labels=Y)
    cost = tf.reduce_mean(cost_i)
    
    train = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cost)
    
    prediction = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    correct_prediction = tf.equal(prediction, Y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))
    
    print("\n============Processing============")
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        summary_writer = tf.summary.FileWriter('./logg', sess.graph)
        for step in range(10001):
            sess.run(train, feed_dict={X: train_x, Y: train_y})
            if step % 1000 == 0:
                loss, acc = sess.run([cost, accuracy], feed_dict={X: train_x, Y: train_y})
                print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))
                
        train_acc = sess.run(accuracy, feed_dict={X: train_x, Y: train_y})
        test_acc,test_predict,test_correct = sess.run([accuracy,prediction,correct_prediction], feed_dict={X: test_x, Y: test_y})
        
        print("\n============Results============")
        print("Model Prediction =", train_acc)
        print("Test Prediction =", test_acc)
        
        return train_acc,test_acc
ann_mlp_train_acc, ann_mlp_test_acc = ann_mlp()

===========Data Summary===========
Training Data : (26920, 53)
Testing Data : (11538, 53)

============Processing============
Step:     0	Loss: 1.039	Acc: 53.75%
Step:  1000	Loss: 0.830	Acc: 55.69%
Step:  2000	Loss: 0.776	Acc: 56.17%
Step:  3000	Loss: 0.748	Acc: 56.82%
Step:  4000	Loss: 0.730	Acc: 57.50%
Step:  5000	Loss: 0.717	Acc: 58.04%
Step:  6000	Loss: 0.708	Acc: 58.50%
Step:  7000	Loss: 0.701	Acc: 59.12%
Step:  8000	Loss: 0.695	Acc: 59.38%
Step:  9000	Loss: 0.690	Acc: 59.56%
Step: 10000	Loss: 0.686	Acc: 59.88%

============Results============
Model Prediction = 0.59877414
Test Prediction = 0.5864968


#### Accuracy using mlp is 58%